In [ ]:
import os
from pymongo import MongoClient
import pandas as pd
import json
from googleapiclient.discovery import build

#### Importing wishlist csv

In [ ]:
wishlist_df = pd.read_csv('./csvs/modified_wish_list_x.csv')     # last updated videos dataset

In [ ]:
wishlist_df.info()

#### Getting list of youtube_id for all wishlist videos in live dataset 

In [ ]:
wishlist_YTids_list = list(wishlist_df['youtube_id'])

### YouTube API key

In [ ]:
developer_key = os.environ.get('DEVELOPER_KEY')

youtube = build('youtube', 'v3', developerKey=developer_key) # project-0127 Data Cleaning


##### Fetching youtube status of wishlist videos

In [ ]:
part_string = ['status', 'contentDetails']
videos_youtube_id = wishlist_YTids_list

In [ ]:
# Considering wishlist videos youtube status available when
#     - embeddable = True
#     - privacy status = public / unlisted
#     - upload status = processed / uploaded

wishlist_youtube_status_dict = {}
for idx in range(0,len(videos_youtube_id)):

    try:
        request = youtube.videos().list(
            part=part_string,
            id=videos_youtube_id[idx],
        )
        response = request.execute()

        if response['items']!=[] and all(key in response['items'][0]['status'] for key in ('embeddable', 'privacyStatus', 'uploadStatus')):
            if response['items'][0]['status']['embeddable'] == True and (response['items'][0]['status']['privacyStatus'] == 'public' or response['items'][0]['status']['privacyStatus'] == 'unlisted') and (response['items'][0]['status']['uploadStatus'] == 'processed' or response['items'][0]['status']['uploadStatus'] == 'uploaded'):

                wishlist_youtube_status = 'available'
        else:
            wishlist_youtube_status = 'unavailable'

    except Exception as e:
        print("An exception occured")
        print(e)
        wishlist_youtube_status = ''

    wishlist_youtube_status_dict[videos_youtube_id[idx]] = wishlist_youtube_status   
        

In [ ]:
youtube_status = open('./youtube_api/wishlist_youtube_status.txt',"a")
youtube_status.write(json.dumps(wishlist_youtube_status_dict))
youtube_status.close()

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to wishlist collection

In [ ]:
wishlist = database['modified_wishlist_x']  # updated wishlist collection

##### Updating youtube_status for wishlist videos

In [ ]:
try:
    for (key,value) in wishlist_youtube_status_dict.items():
        wishlist.update_many(
        {"youtube_id" : key},
        {"$set": { "youtube_status" : value}}
        );
except Exception as e:
    print(e)

#### Drop wishlist videos with youtube status as unavailable

In [ ]:
pipeline = [
    {
        '$match': {
            'youtube_status': 'available'
        }
    }, {
        '$out': 'modified_wish_list_xx'
    }
]

In [ ]:
try: 
    wishlist.aggregate(pipeline)
except Exception as e:
    print(e)